In [ ]:
!pip install emoji

In [ ]:
#Importing Libraries
import pandas as pd
import re

In [ ]:
import re
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import emoji

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#Path to the Feather file on Google Drive
#file_path = '/content/drive/My Drive/output_april.feather'

#df = pd.read_feather(file_path)
#df_final=pd.read_feather(file_path)

In [ ]:
#import os
#os.listdir('Documents')

In [ ]:

# Replace the file path with your Feather file's path
file_path = 'Desktop/output_feb.feather'
file_path2 = 'Desktop/output_mar.feather'
file_path3= 'Desktop/output_april.feather'
file_path4 = 'Desktop/output_may.feather'
file_path5 = 'Desktop/output_jun.feather'
file_path6 = 'Desktop/output_jul.feather'
file_path7 = 'Desktop/output_aug.feather'
file_path8 = 'Desktop/output_sept.feather'
file_path9 = 'Desktop/output_oct.feather'
file_path10 = 'Desktop/output_nov.feather'

dffeb = pd.read_feather(file_path)
dfmar = pd.read_feather(file_path2)
dfapril = pd.read_feather(file_path3)
dfmay = pd.read_feather(file_path4)
dfjun = pd.read_feather(file_path5)
dfjul = pd.read_feather(file_path6)
dfaug = pd.read_feather(file_path7)
dfsep = pd.read_feather(file_path8)
dfoct = pd.read_feather(file_path9)
dfnov = pd.read_feather(file_path10)

In [ ]:

 #Merge all DataFrames into a single DataFrame
df_merged = pd.concat([dffeb, dfmar, dfapril, dfmay, dfjun, dfjul, dfaug, dfsep, dfoct, dfnov], ignore_index=True)



In [ ]:
df_merged.head()

,text,tweetid,userid,date,rt_id,rt_userid
0,@realDonaldTrump leading pool-boy @marcorubio ...,702946164992114688,3291772795,Thu Feb 25 20:00:00 +0000 2016,None,None
1,RT @TheRightScoop: Rush Limbaugh: Ted Cruz fou...,702946165529120772,30080259,Thu Feb 25 20:00:00 +0000 2016,702940427289284609,212708176
2,"RT @realDonaldTrump: ""@HosierN: @foxnewspoliti...",702946168989466624,3702073156,Thu Feb 25 20:00:01 +0000 2016,702680136513155072,25073877
3,@realDonaldTrump Mexico isnt gonna pay for the...,702946170176311296,236730194,Thu Feb 25 20:00:01 +0000 2016,None,None
4,"@realDonaldTrump He may not have beat you, but...",702946171375980544,115698491,Thu Feb 25 20:00:02 +0000 2016,None,None


In [ ]:
unique_userid_count = df_merged['userid'].nunique()
print(unique_userid_count)
# there are 852278 unique users in this dataset.

6719355


In [ ]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74231650 entries, 0 to 74231649
Data columns (total 6 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   text       object
 1   tweetid    object
 2   userid     object
 3   date       object
 4   rt_id      object
 5   rt_userid  object
dtypes: object(6)
memory usage: 3.3+ GB


In [ ]:
len(df_merged)

74231650

In [ ]:
def analyze_dataframe(df):
    """
    Analyzes a DataFrame for null values and counts unique tweet IDs used in 'rt_id'.

    Parameters:
    df (DataFrame): The DataFrame to analyze.

    Returns:
    str: A summary of the analysis.
    """
    # Check for null values in the DataFrame
    null_counts = df.isnull().sum()

    # Count how many tweet IDs are used in rt_id
    tweet_ids_in_rt_id = df['rt_id'].str.extractall(r'(\d+)')[0].nunique()

    summary = "Null Value Counts:\n" + str(null_counts) + "\n\nNumber of tweet IDs used in rt_id: " + str(tweet_ids_in_rt_id)
    return summary


In [ ]:
analyze_dataframe(df_merged)

'Null Value Counts:\ntext                0\ntweetid             0\nuserid              0\ndate                0\nrt_id        23255387\nrt_userid    23255387\ndtype: int64\n\nNumber of tweet IDs used in rt_id: 69412'

In [ ]:
def add_retweet_info(df):
    """
    Adds a retweet count column to the DataFrame and imputes the 'rt_id' and 'rt_userid' columns.

    Parameters:
    df (DataFrame): The DataFrame to process.

    Returns:
    DataFrame: The updated DataFrame with retweet information.
    """
    # Count how many times each tweet ID appears in the rt_id column
    retweet_counts = df['rt_id'].value_counts()

    # Map the count to the tweetid column
    df['retweet_count'] = df['tweetid'].map(retweet_counts).fillna(0)

    # Impute 'rt_id' and 'rt_userid' columns based on the retweet count
    df['rt_id'] = df.apply(lambda x: "there are retweets" if x['retweet_count'] > 0 else "no retweets", axis=1)
    df['rt_userid'] = df.apply(lambda x: "there are retweets" if x['retweet_count'] > 0 else "no retweets", axis=1)

    return df


In [ ]:
#calling funcction
updated_df = add_retweet_info(df_merged)


In [ ]:
unique_userid_count = updated_df['userid'].nunique()
print(unique_userid_count)
# there are  unique users in this dataset.

6719355


In [ ]:
selected_data = updated_df.loc[updated_df['tweetid'] == '699968804311470080', ['text', 'tweetid','userid','date','rt_id','rt_userid','retweet_count']]
selected_data.head(100)
#Once we gone through all the null values, we relaized that we cant remove all the nuill values becuse those are tweets that are not retweed by anyone
#these are still usefull the analysis, so we are goine impute with "No_one_retweeted" on the both columns


,text,tweetid,userid,date,rt_id,rt_userid,retweet_count
3671900,.@FoxNews is so biased it is disgusting. They ...,699968804311470080,25073877,Wed Feb 17 14:49:02 +0000 2016,there are retweets,there are retweets,23870.0


In [ ]:
duplicates = updated_df[updated_df.duplicated()]
duplicates.head()
# The following dataset contians all the duplicates in df dataset
duplicates.count()
# so there are  duplicate in the dataset so we are gone remove these duplicates from the dataset

text             847129
tweetid          847129
userid           847129
date             847129
rt_id            847129
rt_userid        847129
retweet_count    847129
dtype: int64

In [ ]:
filtered_df = updated_df.drop_duplicates()

In [ ]:
duplicates = filtered_df[filtered_df.duplicated()]
duplicates.head()
# The following dataset contians all the duplicates in df dataset
duplicates.count()
# so there are  duplicate in the dataset so we are gone remove these duplicates from the dataset

text             0
tweetid          0
userid           0
date             0
rt_id            0
rt_userid        0
retweet_count    0
dtype: int64

In [ ]:
len(filtered_df)

73384521

In [ ]:

filtered_df = pd.DataFrame(filtered_df)
# Convert the 'date' column into datetime format
filtered_df['datetime'] = pd.to_datetime(filtered_df['date'], format='%a %b %d %H:%M:%S +0000 %Y')
# Extract individual components
filtered_df['day_of_week'] = filtered_df['datetime'].dt.strftime('%A')
filtered_df['month'] = filtered_df['datetime'].dt.strftime('%B')
filtered_df['day'] = filtered_df['datetime'].dt.day
filtered_df['hour'] = filtered_df['datetime'].dt.hour
filtered_df['minute'] = filtered_df['datetime'].dt.minute
filtered_df['second'] = filtered_df['datetime'].dt.second
filtered_df['year'] = filtered_df['datetime'].dt.year
filtered_df['timezone'] = filtered_df['date'].str.split(' ').str[4]
# Drop the original 'date' column and 'datetime' column if not needed
#filtered_df = filtered_df.drop(columns=['date', 'datetime'])

In [ ]:
# Check if a tweet contains a link
filtered_df['contains_link'] = filtered_df['text'].str.contains('http://|https://|www\.')

# Count the tweets with and without links
link_counts = filtered_df['contains_link'].value_counts()

link_counts


contains_link
False    50020967
True     23363554
Name: count, dtype: int64

In [ ]:
def extract_emojis(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F700-\U0001F77F"  # alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           u"\U00002702-\U000027B0"  # Dingbats
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.findall(text)

filtered_df['emojis'] = filtered_df['text'].apply(extract_emojis)
emoji_counts = filtered_df['emojis'].explode().value_counts()

In [ ]:
filtered_df['tweet_length'] = filtered_df['text'].apply(len)

In [ ]:
# Reset the index of the filtered DataFrame if needed
filtered_df.reset_index(drop=True, inplace=True)
# Saving the DataFrame to a Feather file
filtered_df.to_feather('df_all_merged_data_sentimentanalysis.feather')

In [ ]:
rt_count = updated_df['text'].str.startswith('RT').sum()
print("Number of tweets starting with 'RT':", rt_count)

Number of tweets starting with 'RT': 50976326


In [ ]:
# Filter the DataFrame to drop rows where tweets start with "RT"
filtered_df = filtered_df[~filtered_df['text'].str.startswith('RT')]
# Reset the index of the filtered DataFrame if needed
filtered_df.reset_index(drop=True, inplace=True)

In [ ]:
#filtered_df = updated_df
filtered_df.head()

,text,tweetid,userid,date,rt_id,rt_userid,retweet_count,datetime,day_of_week,month,day,hour,minute,second,year,timezone,contains_link,emojis,tweet_length
0,@realDonaldTrump leading pool-boy @marcorubio ...,702946164992114688,3291772795,Thu Feb 25 20:00:00 +0000 2016,no retweets,no retweets,0.0,2016-02-25 20:00:00,Thursday,February,25,20,0,0,2016,+0000,True,[],140
1,@realDonaldTrump Mexico isnt gonna pay for the...,702946170176311296,236730194,Thu Feb 25 20:00:01 +0000 2016,no retweets,no retweets,0.0,2016-02-25 20:00:01,Thursday,February,25,20,0,1,2016,+0000,False,[],137
2,"@realDonaldTrump He may not have beat you, but...",702946171375980544,115698491,Thu Feb 25 20:00:02 +0000 2016,no retweets,no retweets,0.0,2016-02-25 20:00:02,Thursday,February,25,20,0,2,2016,+0000,False,[],69
3,@tedcruz Don't allow Trump say LIAR when He's ...,702946181920514049,523773033,Thu Feb 25 20:00:04 +0000 2016,no retweets,no retweets,0.0,2016-02-25 20:00:04,Thursday,February,25,20,0,4,2016,+0000,False,[],139
4,@realDonaldTrump You know white people all com...,702946189533179904,61365816,Thu Feb 25 20:00:06 +0000 2016,no retweets,no retweets,0.0,2016-02-25 20:00:06,Thursday,February,25,20,0,6,2016,+0000,False,[],85


In [ ]:
len(filtered_df)

23006164

In [ ]:
analyze_dataframe(filtered_df)

'Null Value Counts:\ntext             0\ntweetid          0\nuserid           0\ndate             0\nrt_id            0\nrt_userid        0\nretweet_count    0\ndatetime         0\nday_of_week      0\nmonth            0\nday              0\nhour             0\nminute           0\nsecond           0\nyear             0\ntimezone         0\ncontains_link    0\nemojis           0\ntweet_length     0\ndtype: int64\n\nNumber of tweet IDs used in rt_id: 0'

In [ ]:
# Ensure you have the necessary NLTK data downloaded
nltk.download('wordnet')
nltk.download('stopwords')

def char_is_emoji(character):
    #en_emoji = emoji.UNICODE_EMOJI.get("en")
    #return character[0] in emoji.UNICODE_EMOJI_ENGLISH
    return emoji.is_emoji(character[0])  #check out the API for emoji here: https://carpedm20.github.io/emoji/docs/api.html#

# Let's test it with a couple of examples
# Now we will add a layer to test every character in a string to see if it
# is an emoji.
def text_has_emoji(text):
    for character in text:
        if char_is_emoji(character):
            return True
    return False

text_has_emoji("Python is go!"), text_has_emoji("Python is go 👍")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(False, True)

In [ ]:


slangs = {"afap":"as far as possible", "aab":"average at best", "aak":"alive and kicking", "aamof":"as a matter of fact", "aap":"always a pleasure", "aar":"at any rate", "aayf":"as always: your friend", "abd":"already been done", "admin":"administrator", "adn":"any day now", "aeae":"and ever and ever", "aeap":"as early as possible", "afaiac":"as far as i am concerned", "afaic":"as far as i am concerned", "afaics":"as far as i can see", "afaict":"as far as i can tell", "afaik":"as far as i know", "afair":"as far as i remember", "afc":"always from computer", "agw":"all going well", "afd":"all fucking day", "aft":"about fucking time", "b4":"before", "bb":"bye bye", "bbiab":"be back in a bit", "bbl":"be back later", "bbs":"be back soon", "bd":"big deal", "brb":"be right back", "brt":"be right there", "btw":"by the way", "cu":"see you", "cul":"see you later", "cuz":"because", "cya":"see you", "da":"the", "dat":"that", "der":"there", "dunno":"do not know", "fwiw":"for what it is worth", "fyi":"for your information", "g2g":"got to go", "gtg":"got to go", "gal":"get a life", "gfy":"good for you", "gonna":"going to", "icydk":"in case you do not know", "idgi":"i do not get it", "idk":"i do not know", "ily":"i love you", "ilu":"i love you", "imho":"in my honest opinion", "imo":"in my opinion", "irl":"in real life", "jj":"just joking", "jk":"just kidding", "k":"okay", "kk":"ok cool", "l8r":"later", "lmao":"laughing my ass off", "lmfao":"laughing my fucking ass off", "lol":"laughing out loud", "myob":"mind your own business", "noyb":"none of your business", "np":"no problem", "nsfw":"not safe for work", "nvm":"never mind", "omg":"oh my god", "pic":"picture", "ot":"off topic", "pix":"pictures", "plz":"please", "pls":"please", "ppl":"people", "rofl":"rolling on the floor laughing", "roflmao":"rolling on the floor laughing my ass off", "roflmaol":"rolling on the floor laughing my ass out loud", "smh":"shaking my head", "thnx":"thanks", "tho":"though", "tia":"thanks in advance", "ttyl":"talk to you later", "ttyt":"talk to you tomorrow", "ty":"thank you", "u":"you", "w8":"wait", "wanna":"want to", "wb":"welcome back", "wd":"well done", "im":"i am", "you've":"you have", "how'd":"how did", "i'm":"i am", "n't":"not", "we're":"we are", "you're":"you are"}
negated = {"weren't":"were not", "hadn't": "had not", "wouldn't": "would not", "mustn't": "must not", "aren't": "are not", "wasn't": "was not", "couldn't": "could not", "shan't": "shall not", "haven't": "have not", "mightn't": "might not", "doesn't": "does not", "needn't": "need not", "don't": "do not", "isn't": "is not", "hasn't": "has not", "doesn't": "does not", "didn't": "did not", "won't": "will not", "oughtn't": "ought not", "shouldn't": "should not", "cannot": "can not", "can't": "can not"}
emoticons = ["[: )", ":-]", ":-3", ":->", "8-)", ":-}", ":)", ":]", ":3", ":>", "8)", ":}", ":o)", ":c)", ":^)", "=]", "=)", ":-))", ": D", "8 D", "x D", "X D", ":D", "8D", "xD", "XD", "=D", "=3", "B^D", ":-*", ":*", ":×'", "; )", "*-)", "; ]", ";)", "*)", ";]", ";^)", ": ,", ";D", "<3", "(:", ": (", ": c", ": <", ": [", ":(", ":c", ":<", ":[", ":-||", ">:[", ":{", ":@", ">:(", ":' (", ":'(", "D ':", "D:<", "D:", "D8", "D;", "D=", "DX", ": O", ": o", ":O", ":o", ":-0", "8 0", ">:O"]

type(slangs), type(negated), type(emoticons)

(dict, dict, list)

In [ ]:
# Next we will use our emoji finder to clean up the texts of a list of tweets.
# This function tokenizes the input and does many different transformations
# to the text and the tokens.



def clean_tweets(text):
        cleaned_tweets ='' # This will be our return list
        hash_emos = [] # This is an overall list of emoticons and emojis

    # Loop over the full list of texts provided
          # Simulating a delay
        hash_emo = [] # Initialize an empty list for this one text

        # These make substitutions of short tags in place of repeating
        # exclamation marks and/or question marks.
        text = re.sub('(!){2,}', ' <!repeat> ', text)
        text = re.sub('(\?){2,}', ' <?repeat> ', text)

        # Tokenize using tweet tokenizer: This will treat each emoji as
        # its own token.
        tokenizer = nltk.tokenize.TweetTokenizer(strip_handles=False, reduce_len=True)
        tokens = tokenizer.tokenize(text.lower())
        lemmatizer = WordNetLemmatizer()


        # Emojis and emoticons
        if text_has_emoji(text): # Examine the untokenized text
            temp = [] # Make temporary storage for the modified tokens
            for tok in tokens: # Cycle through each token in this text
                # If the token is an emoji, look up the equivalent
                # plain text placeholder.
                if char_is_emoji(tok):
                    temp.append(emoji.demojize(tok))
                elif tok in emoticons:
                    temp.append(tok)
                else:
                    temp.append(tok)
            tokens = temp

        # Hashtags
        temp = []
        for word in tokens:
            if '#' in word: # Do a simple substitution using the empty string
                word = word.replace('#','')
                temp.append(word)
            else:
                temp.append(word)
        tokens = temp

        # Replace slangs and negated words
        temp = []
        for word in tokens:
            if word in slangs:
                temp += slangs[word].split()

            elif word in negated:
                temp += negated[word].split()

            else:
                temp.append(word)

        tokens = temp

        # Replace user names
        tokens = ['<user>'  if '@' in word else word for word in tokens]

        #Replace numbers
        tokens = ['<number>' if word.isdigit() else word for word in tokens]

        # Remove urls
        tokens = ['' if 'http' in word else word for word in tokens]

        # Lemmatize
        #tokens = [lemmatizer.lemmatize(word) for word in tokens]

        # Remove stop words
        tokens = [word for word in tokens if word not in nltk.corpus.stopwords.words('english')]

        # Remove tokens having length 1
        tokens = [word for word in tokens if word != '' and len(word) > 1]

        cleaned_tweets=tokens

        return ' '.join(cleaned_tweets)

In [ ]:
df_final_1=filtered_df
#start_time = time.time()
df_final_1["clean_data"]=df_final_1['text'].apply(clean_tweets)

#end_time = time.time()  # End time
#total_time = end_time - start_time

#print(f"The function took {total_time/60} seconds to run.")

In [ ]:
df_final_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23006164 entries, 0 to 23006163
Data columns (total 20 columns):
 #   Column         Dtype         
---  ------         -----         
 0   text           object        
 1   tweetid        object        
 2   userid         object        
 3   date           object        
 4   rt_id          object        
 5   rt_userid      object        
 6   retweet_count  float64       
 7   datetime       datetime64[ns]
 8   day_of_week    object        
 9   month          object        
 10  day            int32         
 11  hour           int32         
 12  minute         int32         
 13  second         int32         
 14  year           int32         
 15  timezone       object        
 16  contains_link  bool          
 17  emojis         object        
 18  tweet_length   int64         
 19  clean_data     object        
dtypes: bool(1), datetime64[ns](1), float64(1), int32(5), int64(1), object(11)
memory usage: 2.8+ GB


In [ ]:
df_final_1.head()

,text,tweetid,userid,date,rt_id,rt_userid,retweet_count,datetime,day_of_week,month,day,hour,minute,second,year,timezone,contains_link,emojis,tweet_length,clean_data
0,@realDonaldTrump leading pool-boy @marcorubio ...,702946164992114688,3291772795,Thu Feb 25 20:00:00 +0000 2016,no retweets,no retweets,0.0,2016-02-25 20:00:00,Thursday,February,25,20,0,0,2016,+0000,True,[],140,<user> leading pool-boy <user> home state flor...
1,@realDonaldTrump Mexico isnt gonna pay for the...,702946170176311296,236730194,Thu Feb 25 20:00:01 +0000 2016,no retweets,no retweets,0.0,2016-02-25 20:00:01,Thursday,February,25,20,0,1,2016,+0000,False,[],137,<user> mexico isnt going pay fucking wall ... ...
2,"@realDonaldTrump He may not have beat you, but...",702946171375980544,115698491,Thu Feb 25 20:00:02 +0000 2016,no retweets,no retweets,0.0,2016-02-25 20:00:02,Thursday,February,25,20,0,2,2016,+0000,False,[],69,<user> may beat still loser
3,@tedcruz Don't allow Trump say LIAR when He's ...,702946181920514049,523773033,Thu Feb 25 20:00:04 +0000 2016,no retweets,no retweets,0.0,2016-02-25 20:00:04,Thursday,February,25,20,0,4,2016,+0000,False,[],139,<user> allow trump say liar he's real devious ...
4,@realDonaldTrump You know white people all com...,702946189533179904,61365816,Thu Feb 25 20:00:06 +0000 2016,no retweets,no retweets,0.0,2016-02-25 20:00:06,Thursday,February,25,20,0,6,2016,+0000,False,[],85,<user> know white people come native african b...


In [ ]:
# Resetting the index of the DataFrame
df_final_1.reset_index(drop=True, inplace=True)

# Saving the DataFrame to a Feather file
df_final_1.to_feather('df_all_merged&filtered_cleaned_data_topiclabels.feather')


In [ ]:
len(df_final_1)

23006164